# Import

In [ ]:
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

import pickle
#from google.colab import drive

#drive.mount("/content/gdrive")#, force_remount=True)


pd.set_option('display.max_columns', None)

pd.set_option('mode.chained_assignment', None)

#ovo je za velicinu slika
sns.set(rc={'figure.figsize':(20,10)})

#import warnings
#warnings.filterwarnings('ignore')

In [ ]:
cd "C:\Users\Ivan Emanuel Pavlov\OneDrive - Prirodoslovno-matematički fakultet\Github\LUMEN2021"


In [ ]:
data=pd.read_csv(r"LUMEN_DS_21.csv",sep=",")


# Data cleaning

## Dates

In [ ]:
#def days(x):
    #return x.days

def months(x):
    return np.round(x.days/30)

#def years(x):
 #   return x.years

date_names=['Customer First Invoice Date','Price last modified date in the ERP',
           'Invoice Date', 'Order Date','Born on date']
#month_names=[]


zero_time=pd.to_datetime('2018-01-01',format='%Y-%m-%d')
#zero_time=zero_time.month

for i in date_names:
    data[i]=pd.to_datetime(data[i],format='%Y-%m-%d', errors='coerce') 
    data[i+' Months']= (data[i]-zero_time).apply(months)
    

data["Invoice Date Months"]=data["Invoice Date Months"]+1

data["Invoice Date Months u godini"]=(data["Invoice Date Months"]%12+1)

data["Ordered Date Months u godini"]=(data["Order Date Months"]%12+1)

## Sale channel grouping drop

In [ ]:
data=data.drop(['Sales Channel - Internal', 'Sales Channel - External','Sales Channel - Grouping'],axis=1)

data=data.drop(['Order #', 'Order Line #'],axis=1)

data=data.drop(['Material cost of part','Labor cost of part', 'Overhead cost of part'],axis=1)

## Price outliers drop

In [ ]:
#dropanje cjenovnih outliera
data=data.drop([594755,89287,826942,1154597],axis=0)


## GM range reduction


In [ ]:
data=data[(data["GM%"]<1)|(data["GM%"].isna())]
data=data[(data["GM%"]>0.1)|(data["GM%"].isna())]




# Encoding

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()



In [ ]:
df2=data

df2=df2.dropna(how="any")


df2["Log Invoiced qty (shipped)"]=df2["Invoiced qty (shipped)"].apply(np.log10).replace([np.inf, -np.inf], np.nan)
df2["Log Ordered qty"]=df2["Ordered qty"].apply(np.log10).replace([np.inf, -np.inf], np.nan)
df2["Log Cost of part"]=df2["Cost of part"].apply(np.log10).replace([np.inf, -np.inf], np.nan)


#df2["Log Material of cost of part"]=df2["Material cost of part"].apply(np.log10).replace([np.inf, -np.inf], np.nan)
#df2["Log Labor cost of part"]=df2["Labor cost of part"].apply(np.log10).replace([np.inf, -np.inf], np.nan)
#df2["Log Overhead cost of part"]=df2["Overhead cost of part"].apply(np.log10).replace([np.inf, -np.inf], np.nan)





#transform po varijablama

df2["Manufacturing Region"] = le.fit_transform(df2["Manufacturing Region"])
df2["Manufacturing Location Code"] = le.fit_transform(df2["Manufacturing Location Code"])
df2["Intercompany"] = le.fit_transform(df2["Intercompany"])
df2["Customer industry"] = le.fit_transform(df2["Customer industry"])
df2["Customer Region"] = le.fit_transform(df2["Customer Region"])
df2["Top Customer Group"] = le.fit_transform(df2["Top Customer Group"])
df2["Product family"] = le.fit_transform(df2["Product family"])
df2["Top Customer Group"] = le.fit_transform(df2["Top Customer Group"])
df2["Product group"] = le.fit_transform(df2["Product group"])
df2["Make vs Buy"] = le.fit_transform(df2["Make vs Buy"])
#df2["Sales Channel - Internal"] = le.fit_transform(df2["Sales Channel - Internal"])

df2=df2.dropna(how="any")

# Chaid tree

In [ ]:
#Pomoću chaid stabla segmentiramo dealove

In [ ]:

def getSplits(ddf, Rules):
  df_split = []
  for k in Rules:
    v = True
    for rules in k['rules']:
      vv = False
      for data in rules['data']:
        vv |=ddf[rules['variable']]== data
      v &= (vv)
    #print(v)
    df_split.append(v)
  return df_split



independent_variable_columns = [  'Customer Region',
                                'Customer industry', 'Top Customer Group','Manufacturing Location Code', 
                                'Make vs Buy', 'Product group',"Order Date Months"]
dep_variable = 'GM%'
from CHAID import Tree
treeCustomer = Tree.from_pandas_df(df2, dict(zip(independent_variable_columns, ['nominal'] * 6)), dep_variable, dep_variable_type='continuous', max_depth=8, min_parent_node_size=500, min_child_node_size = 100)



#treeCustomer.render(path='Tree', view=True)
CustomerRules = treeCustomer.classification_rules()



df_split = getSplits(df2, CustomerRules)
#for k in df_split:
#  plt.figure()
#  df2.loc[k]['GM%'].hist()

# Kmeans


In [ ]:
df3=df2

df3=df3.dropna(how="any")

In [ ]:
from sklearn.cluster import KMeans
from kneed import KneeLocator, DataGenerator


In [ ]:
def inercije():
    inertias=[]
    K = [i for i in range(1,15)]
    for num_clusters in K:
        kmeans = KMeans(n_clusters=num_clusters, init = "k-means++", n_init = 3, verbose=0, n_jobs=-1)
        kmeans.fit_predict(a[["Log Ordered qty","Log Cost of part"]]) 
        inertias.append(kmeans.inertia_)
    kneedle = KneeLocator(K, inertias, S=1.0, curve='convex', direction='decreasing')

    return kneedle.elbow

In [ ]:

def segmentacija_po_kontinuiranim_varijablama():
    ukupni_len_prijasnih_clustera=0
    for j in range(len(df_split)):
       
        a=df3.loc[df_split[j]]
    
        elbow=inercije() 
    
        kmeans = KMeans(n_clusters=elbow, random_state=j,init = "k-means++", n_init = 10, verbose=0, n_jobs=-1).fit(a[["Log Ordered qty","Log Cost of part"]])

        y_kmeans = kmeans.predict(a[["Log Ordered qty","Log Cost of part"]])
    
        df3.loc[df_split[j],"clusters"]=(y_kmeans+ukupni_len_prijasnih_clustera).astype(int)
 
        ukupni_len_prijasnih_clustera=ukupni_len_prijasnih_clustera+elbow


#zapisujemo clustere koji su premali za segmentaciju po kvantilima. Njih cemo reflasificifati stablom odlučivanja   
premali=[]


def dodjeljivanje_ocjena_i_margina():
    for j in range(len(df3.clusters.unique())):   
        #print(j)
    
        try:
            
            df3.loc[df3.clusters==j,"kvantili"]=pd.qcut(df3.loc[df3.clusters==j,"GM%"],q=5,labels=[str(pd.qcut(df3.loc[df3.clusters==j,"GM%"],q=5,precision=2).sort_values().unique()[i]) for i in range(5)])
    
            df3.loc[df3.clusters==j,"ocjene"]=pd.qcut(df3.loc[df3.clusters==j,"GM%"],q=5,labels=["F","D","C","B","A"])       
            #break   
             
        except:
                premali.append(j)
                j=j+1
                #continue    

            
#odvajamo anomalne clustere

anomalni_clusteri=df3[df3.clusters.isin(premali)]

# Decision tree clasificatior

In [ ]:
import os
os.environ["PATH"] += os.pathsep + r"C:\Program Files\Graphviz\bin"

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split


#definiramo klasifikacijsku preciznost


def accuracy(X_test,y_test):
    tocni=0
    krivi=0
    
    pred=tree_clf.predict(X_test)
    
    lista=(pred-np.array(y_test))
    for i in range(len(lista)):
        if lista[i]==0:
            tocni=tocni+1
        else:
            krivi=krivi+1
    ukupno=krivi+tocni
    acc=tocni/(ukupno)
    return acc        


#iz df3 dataseta izbacujemo zasada dealove iz anomalnih clustera čije ćemo nove clustere potom predictati s istreniranim klasifikatorom te ponovno umetnuti u df3 te će sada pripadati
#neanomalnim clusterima 



X_train, X_test, y_train, y_test = train_test_split(df3.drop(anomalni_clusteri.index)[['Manufacturing Region', 'Manufacturing Location Code',
       'Intercompany','Customer industry', 'Customer Region','Top Customer Group',
       'Product family', 'Product group','Make vs Buy','Invoiced qty (shipped)', 'Ordered qty','# of unique products on a quote','Customer First Invoice Date Months',
       'Price last modified date in the ERP Months', 'Invoice Date Months',
       'Order Date Months', 'Born on date Months',
       'Invoice Date Months u godini', 'Ordered Date Months u godini',
       'Log Invoiced qty (shipped)', 'Log Ordered qty', 'Log Cost of part']], df3.drop(anomalni_clusteri.index).clusters, test_size=0.10, random_state=50,shuffle=True)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10,random_state=80,shuffle=True) # 0.25 x 0.8 = 0.2

tree_clf = DecisionTreeClassifier(max_depth=None) 
tree_clf.fit(X_train, y_train)


print("preciznost klasifikatora=",accuracy(X_test,y_test))

#predictamo nove clustere za dealove iz "anomalnih" clustera

y_anomalni=tree_clf.predict(anomalni_clusteri[['Manufacturing Region', 'Manufacturing Location Code',
       'Intercompany','Customer industry', 'Customer Region','Top Customer Group',
       'Product family', 'Product group','Make vs Buy','Invoiced qty (shipped)', 'Ordered qty','# of unique products on a quote','Customer First Invoice Date Months',
       'Price last modified date in the ERP Months', 'Invoice Date Months',
       'Order Date Months', 'Born on date Months',
       'Invoice Date Months u godini', 'Ordered Date Months u godini',
       'Log Invoiced qty (shipped)', 'Log Ordered qty', 'Log Cost of part']])


#umećemo nove clustere na mjesta gdje su prije bili anomalni clusteri
df3.loc[anomalni_clusteri.index,"clusters"]=y_anomalni


#spremamo istrenirani klasifikator
filename = 'stablo_odlucivanja_3.sav'
pickle.dump(tree_clf, open(filename, 'wb'))

#spremamo segmentirani dataset
df3.to_csv(r'data_za_model.csv',index=False)